In [1]:
%matplotlib inline

import sys
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sc
from scipy import signal

In [2]:
# Para que los gráficos se vean bien
plt.rcParams["figure.figsize"] = (12, 6)
plt.rcParams["axes.grid"] = True

In [ ]:
csv_path = "Appendix 1 - xy_motion_kalman_filter_example.csv"
df = pd.read_csv(csv_path, header=0)
df.columns = ["x_meas", "y_meas"]
z = df[["x_meas","y_meas"]].values.astype(float)

vx0 = z[1,0] - z[0,0] if len(z) >= 2 else 0.0
vy0 = z[1,1] - z[0,1] if len(z) >= 2 else 0.0
x0  = np.array([z[0,0], vx0, z[0,1], vy0], dtype=float)
P0  = np.diag([10.0, 10.0, 10.0, 10.0]).astype(float)
print(x0, P0)

[ 3.55 -0.55  1.8   1.1 ] [[10.  0.  0.  0.]
 [ 0. 10.  0.  0.]
 [ 0.  0. 10.  0.]
 [ 0.  0.  0. 10.]]


In [16]:
dt = 1.0
F = np.array([[1, dt, 0,  0],
              [0,  1, 0,  0],
              [0,  0, 1, dt],
              [0,  0, 0,  1]], dtype=float)

def Q_cv(q_var, dt=1.0):
    dt2, dt3, dt4 = dt**2, dt**3, dt**4
    return q_var*np.array([[dt4/4, dt3/2,   0,     0],
                           [dt3/2, dt2,     0,     0],
                           [0,     0,     dt4/4, dt3/2],
                           [0,     0,     dt3/2, dt2 ]], dtype=float)

qA, rA = 0.05, 0.05
Q_A = Q_cv(qA, dt)
R_A = np.eye(2) * rA

x_pred = F @ x0
P_pred = F @ P0 @ F.T + Q_A
print(x_pred, "\n", P_pred)

[ 3.   -0.55  2.9   1.1 ] 
 [[20.0125 10.025   0.      0.    ]
 [10.025  10.05    0.      0.    ]
 [ 0.      0.     20.0125 10.025 ]
 [ 0.      0.     10.025  10.05  ]]


In [19]:
H = np.array([[1,0,0,0],
              [0,0,1,0]], dtype=float)
S = H @ P_pred @ H.T + R_A
K = P_pred @ H.T @ np.linalg.inv(S)
K

array([[0.99750779, 0.        ],
       [0.49968847, 0.        ],
       [0.        , 0.99750779],
       [0.        , 0.49968847]])

In [24]:
z_t = z[0]
y = z_t - (H @ x_pred)
I = np.eye(4)
x_upd = x_pred + K @ y
P_upd = (I - K @ H) @ P_pred
print(y, "\n", P_upd)

[ 0.55 -1.1 ] 
 [[0.04987539 0.02498442 0.         0.        ]
 [0.02498442 5.04062305 0.         0.        ]
 [0.         0.         0.04987539 0.02498442]
 [0.         0.         0.02498442 5.04062305]]


In [25]:
#q=0.5>r=0.05
def kf_step(x, P, z_t, F, H, Q, R):
    I = np.eye(4)
    x_pred = F @ x
    P_pred = F @ P @ F.T + Q
    y = z_t - (H @ x_pred)
    S = H @ P_pred @ H.T + R
    K = P_pred @ H.T @ np.linalg.inv(S)
    x_upd = x_pred + K @ y
    P_upd = (I - K @ H) @ P_pred
    return x_pred, P_pred, K, y, x_upd, P_upd

def run_kf_series(z, F, H, q_var, r_var, x0, P0, dt=1.0):
    Q, R = Q_cv(q_var, dt), np.eye(2)*r_var
    n = len(z)
    xs = np.zeros((n,4)); Ps = np.zeros((n,4,4))
    innovs = np.zeros((n,2))
    x, P = x0.copy(), P0.copy()
    for t in range(n):
        x_pred, P_pred, Kt, y_t, x, P = kf_step(x, P, z[t], F, H, Q, R)
        xs[t], Ps[t], innovs[t] = x, P, y_t
    return xs, Ps, innovs

xs_B, Ps_B, innov_B = run_kf_series(z, F, H, 0.5, 0.05, x0, P0, dt=1.0)
res_x_B = z[:,0] - xs_B[:,0]
res_y_B = z[:,1] - xs_B[:,2]
print("res_x mean/std:", res_x_B.mean(), res_x_B.std())
print("res_y mean/std:", res_y_B.mean(), res_y_B.std())
print("innov_x mean/std:", innov_B[:,0].mean(), innov_B[:,0].std())
print("innov_y mean/std:", innov_B[:,1].mean(), innov_B[:,1].std())


res_x mean/std: 0.0006405061263257324 0.036958700567728195
res_y mean/std: -0.0008060632488082677 0.02827502682648443
innov_x mean/std: 0.008307570732296665 0.39896737308241903
innov_y mean/std: -0.011421957550428944 0.31537476385883806


In [26]:
var_x_B = np.array([Ps_B[t,0,0] for t in range(len(Ps_B))])
print("Var(x) initial, final:", float(var_x_B[0]), float(var_x_B[-1]))
print("Innovation mean/std (x,y):",
      (float(innov_B[:,0].mean()), float(innov_B[:,0].std())),
      (float(innov_B[:,1].mean()), float(innov_B[:,1].std())))

Var(x) initial, final: 0.04987608426270301 0.04534076289364882
Innovation mean/std (x,y): (0.008307570732296665, 0.39896737308241903) (-0.011421957550428944, 0.31537476385883806)


In [27]:
from scipy.signal import savgol_filter
win = 21; poly = 3
truth_x = savgol_filter(z[:,0], window_length=win, polyorder=poly, mode='interp')
truth_y = savgol_filter(z[:,1], window_length=win, polyorder=poly, mode='interp')
truth = np.column_stack([truth_x, truth_y])

def rmse(a, b):
    return float(np.sqrt(np.mean((np.asarray(a)-np.asarray(b))**2)))


In [28]:
scenarios = {
    "A_all_close": {"q": 0.05, "r": 0.05},   # medición ≈ KF ≈ truth
    "B_all_diff":  {"q": 0.8,  "r": 0.2 },   # los tres distintos
    "C_kf_diff":   {"q": 1e-4, "r": 1.0 },   # meas ≈ truth pero KF distinto
}

rows = []
for name, prm in scenarios.items():
    xs, Ps, innov = run_kf_series(z, F, H, prm["q"], prm["r"], x0, P0, dt=dt)
    x_kf, y_kf = xs[:,0], xs[:,2]
    rows.append({
        "scenario": name,
        "q": prm["q"], "r": prm["r"],
        "RMSE_x(KF,truth)": rmse(x_kf, truth[:,0]),
        "RMSE_y(KF,truth)": rmse(y_kf, truth[:,1]),
        "RMSE_x(meas,truth)": rmse(z[:,0], truth[:,0]),
        "RMSE_y(meas,truth)": rmse(z[:,1], truth[:,1]),
        "res_x_mean": float((z[:,0]-x_kf).mean()),
        "res_x_std":  float((z[:,0]-x_kf).std()),
        "res_y_mean": float((z[:,1]-y_kf).mean()),
        "res_y_std":  float((z[:,1]-y_kf).std()),
    })
summary_e2 = pd.DataFrame(rows)
summary_e2

,scenario,q,r,"RMSE_x(KF,truth)","RMSE_y(KF,truth)","RMSE_x(meas,truth)","RMSE_y(meas,truth)",res_x_mean,res_x_std,res_y_mean,res_y_std
0,A_all_close,0.0500,0.05,0.614451,0.485994,0.562919,0.446344,0.003041,0.116000,-0.004189,0.090516
1,B_all_diff,0.8000,0.20,0.586752,0.465482,0.562919,0.446344,0.001210,0.059028,-0.001545,0.045646
2,C_kf_diff,0.0001,1.00,2.465235,2.301852,0.562919,0.446344,-0.095359,2.572497,-0.530160,2.290363
